In [2]:
import numpy as np

In [3]:
# pongo esto para probar 
c_orig= [62,  18, -34, -88, -97, -24,  19, -15,  31, -13, -63,  81,  -3,  63,   0,   0,   0,   0,   0,   0,] 

A = np.array([
    [-42, -35,  68,  50,  73,  64, -44,  -4,  36, -45, -89,  88,  75, -44,  0,  0,  0,  0,  0,  0], 
    [ 49, -51,  32,  73,  32, -96,  33,  20,  61, -77,  97,   4, -56,   3,  0,  0,  0,  0,  0,  0], 
    [ 21, -26, -86, -68,  31,   1, -10,  84,  13, -62,  -1,  10,  68,  96,  0,  0,  0,  0,  0,  0], 
    [ 49,  92, -39,  66, -89,  -5,  95,  55, -24,  71,   7,  14,  23,  70,  0,  0,  0,  0,  0,  0], 
    [ 73,  61,  93,  96,  66,  99,  76,  53,  78,  55,  91,  68,  68,  54,  1,  0,  0,  0,  0,  0], 
    [-29,  17, -42,  94, -60,  28,  21,  74,  99,  31, -15,   6, -29, -15,  0,  1,  0,  0,  0,  0], 
    [-97,  51, -91,  41, -57,  16,  92,  81, -35,  36,  72,  -6,  22,  14,  0,  0,  1,  0,  0,  0], 
    [-76,  27,  17,  69,  80, -51,   5,  65, -87,   6, -72,  20, -78,  88,  0,  0,  0, -1,  0,  0], 
    [ 69,  -1,  44,  20, -81,  27,  71, -41,  85,  91, -58,   7, -41, -30,  0,  0,  0,  0,  1,  0], 
    [ -5,  87, -60,  15,  94,  65,  76,  69,  62, -20, -14,  68,  53,  96,  0,  0,  0,  0,  0, -1]
])

b= np.array([151 , 124,   71,  385, 1032,  181,  140,   12,  163 , 585])


In [4]:
def añadir_variables_artificiales(A, b):
    vars = []
    for v in b:
        if v >= 0:
            vars.append(1)
        else:
            vars.append(-1)
    array = np.array(vars)
    # Crear una matriz diagonal
    matriz_diagonal = np.diag(array)
    # Usamos np.hstack para apilar horizontalmente
    resultado = np.hstack((A, matriz_diagonal))
    return resultado

r = añadir_variables_artificiales(A, b)

In [ ]:
""" <pseudo-codigo Todo el programa >

def simplexII (A, b, c, B_ini):
    bool_acotado = True
    n var, m eq,
    N_ini = 1..n not in B_ini
    x_i = [0...0]
    x_b = B^-1 b 
    poner x_b  en x_i segun los indices de B_ini 
    bucle iteraciones del simplex, while seguramente
        B = columna iessima de A bucle por A i s en B_ini
        An =
        cn = de c
        cb = 
        algebra lineal, calcular r, con bland miras la q
        d descens -B -1 Aq
        tethas i la variable que sale p 
        buscas q en N_ini, y p en B_ini, y las intercambias
    
        cuando acaba
        - d >= 0 no acotado 
        - costes >= 0 acaba estas en optimo, return

    return B_optima, z , bool_acotado

    
def simplexDefinitu (A,b,c):
    n variables ,m eq
    A_inicial = A + variables 1s o -1 depende de b 
    b_inicial = b 
    c_inicial = [0,0,0,0, 1,1,1]
    B_inicial = [n,n+m]
    B_ini_II, z_inicial, _ = simplexII(A_inicial,b_inicial,c_inicial,B_inicial)
    if (z_inicial != 0):
        return None,None,None,False #sera no factible
    B_optima, z , bool_acotado = simplexII(A,b,c,B_ini_II)

    return B_optima, z , bool_acotado, True (si es fatible o no)
"""

In [37]:
def SimplexFaseII(A, b, c, B_ini, B_indices):
    bool_acotado = True
    m,n = A.shape
    N_ini = set(i for i in range(n) if i not in B_indices) # Antes:(n+1)
    x_ini = [0]*n
    x_b = np.linalg.inv(B_ini) @ b # con @ se multiplican matrices en numpy
    j = 0 #para añadir en orden los x_b
    for i in B_indices:
        x_ini[i] = x_b[j]
        j+=1
    while True:#####################################################################################
        B_indices = sorted(B_indices)
        B_may = A[:, B_indices] # le he cambiado el nombre porque si no era una constante
        N_ini = sorted(N_ini)
        A_n = A[:, N_ini]
        cn = np.array([c[i] for i in N_ini])
        cb = np.array([c[i] for i in B_indices])
        z = cb @ x_b
        B_inversa = np.linalg.inv(B_may)
        #costes reducidos
        r = cn - cb @ B_inversa @ A_n # r corresponde a las variables de los indices N_ini
        if np.all(r >= 0):
            return B_indices, z, bool_acotado, B_may
        i = 0
        minimo = 0
        for j in range(len(r)):
            if r[j] < minimo:
                minimo = r[j]
                i = j
            #elif r[j] == minimo: # Deberiamos elegir el del indice menor pero como N_ini esta ordenado ya habriamos elegido el del menor indice
        q = N_ini[i] # variable q tiene que entrar en la base 
        #direccion de descenso 
        d_b = -B_inversa @ A[:, q]
        if np.all(d_b >= 0):
            bool_acotado = False
            return None, None, bool_acotado, None
        
        #tetha y variable de salida
        p = 0
        theta = float('inf')
        for w in range(len(d_b)):
            if d_b[w] < 0:
                val = -x_b[w]/d_b[w]
                if val < theta:
                    theta, p = val, w
        
        #Actualizaciones
        x_b = x_b + theta*d_b
        z = z + theta * r[i]
        var_saliente, B_indices[p] = B_indices[p], q
        N_ini.remove(q) 
        N_ini.append(var_saliente)
        

        

                    

        
    

In [32]:
def SimplexFaseI(A, b, c):
    m, n = A.shape #las filas son m (el numero de restr.) y n son las columnas (num de variables)
    A_inicial = añadir_variables_artificiales(A, b)
    b_inicial = b 
    c_inicial = [0]*n + [1]*m # ojo q sale vertical ! hay q trasponer??
    B_inicial = A_inicial[:,n:]
    B_indices = set(i for i in range(n, n+m)) #set para que sea mas eficiente buscar indices
    B_indices_II, z_inicial, _,B_ini_II  = SimplexFaseII(A_inicial,b_inicial,c_inicial,B_inicial, B_indices)
    if (z_inicial != 0):
        return None,None,None,False #sera no factible
    
    B_indices_opt, z , bool_acotado, B_optima = SimplexFaseII(A,b,c,B_ini_II,B_indices_II)

    return B_optima, z , bool_acotado, True



Prueba del algoritmo:

In [38]:
c_prueba = [-1,-2,0,0]
A_prueba = np.array([[2,1,1,0],[1,1,0,1]])
b_prueba = np.array([3,2])
opt, zeta, acotado, bool_res = SimplexFaseI(A_prueba,b_prueba,c_prueba)
print(opt,zeta,acotado,bool_res)

[[1 1]
 [1 0]] 0.0 True True


A partir de aqui son pruebas para ver si funciona:

In [8]:
m, n = A.shape
m, n

(10, 20)

In [9]:
B_ind = set(i for i in range(n, n+m))
print(B_ind)
N_ini = set(i for i in range(n + 1) if i not in B_ind)
print(N_ini)

{20, 21, 22, 23, 24, 25, 26, 27, 28, 29}
{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19}


In [10]:
c_incial = [0]*n + [1]*m
c_incial

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1]

In [11]:
B_indices = set(i for i in range(n, n+m))
N_ini = set(i for i in range(n + 1) if i not in B_indices)
B_i = r[:, n:]
print(B_i)
np.linalg.inv(B_i)

[[1 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 0 1]]


array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]])

In [12]:
b

array([ 151,  124,   71,  385, 1032,  181,  140,   12,  163,  585])

In [13]:
m, n = r.shape


In [14]:
x_ini = [0]*n
len(x_ini)

30

In [15]:

x_b =  np.dot(np.linalg.inv(B_i), b)
j = 0 #para añadir en orden los x_b
for i in B_indices:
    x_ini[i] = x_b[j]
    j+=1
    
print(x_ini)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, np.float64(151.0), np.float64(124.0), np.float64(71.0), np.float64(385.0), np.float64(1032.0), np.float64(181.0), np.float64(140.0), np.float64(12.0), np.float64(163.0), np.float64(585.0)]


In [17]:
B_indices = sorted(B_indices)
B = r[:, B_indices]
N_ini = sorted(N_ini)
A_n = r[:, N_ini]
cn = np.array([c_incial[i] for i in N_ini])
cb = np.array([c_incial[i] for i in B_indices])
print(N_ini)
print(A_n)


[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
[[-42 -35  68  50  73  64 -44  -4  36 -45 -89  88  75 -44   0   0   0   0
    0   0]
 [ 49 -51  32  73  32 -96  33  20  61 -77  97   4 -56   3   0   0   0   0
    0   0]
 [ 21 -26 -86 -68  31   1 -10  84  13 -62  -1  10  68  96   0   0   0   0
    0   0]
 [ 49  92 -39  66 -89  -5  95  55 -24  71   7  14  23  70   0   0   0   0
    0   0]
 [ 73  61  93  96  66  99  76  53  78  55  91  68  68  54   1   0   0   0
    0   0]
 [-29  17 -42  94 -60  28  21  74  99  31 -15   6 -29 -15   0   1   0   0
    0   0]
 [-97  51 -91  41 -57  16  92  81 -35  36  72  -6  22  14   0   0   1   0
    0   0]
 [-76  27  17  69  80 -51   5  65 -87   6 -72  20 -78  88   0   0   0  -1
    0   0]
 [ 69  -1  44  20 -81  27  71 -41  85  91 -58   7 -41 -30   0   0   0   0
    1   0]
 [ -5  87 -60  15  94  65  76  69  62 -20 -14  68  53  96   0   0   0   0
    0  -1]]


In [111]:
cb

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [113]:
r = cn - cb @ np.linalg.inv(B) @ A_n
print(r)

[ -12. -222.   64. -456.  -89. -148. -415. -456. -288.  -86.  -18. -279.
 -105. -332.   -1.   -1.   -1.    1.   -1.    1.]


In [116]:
len(r)

20

In [115]:
z = cb @ x_b
z

np.float64(2844.0)